# Preparing our Keras model for serving

In [5]:
import pandas as pd
print(pd.__version__)

0.23.4


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.10.0


In [7]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [8]:
# a small sane check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


In [9]:
from tensorflow import keras
print(keras.__version__)

2.1.6-tf


## Loading and validating our model

In [13]:
!curl -O https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/model/insurance.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  175k  100  175k    0     0   467k      0 --:--:-- --:--:-- --:--:--  467k


In [14]:
!ls -l insurance.h5

-rw-r--r-- 1 olive 197609 179704 Sep 14 09:35 insurance.h5


In [15]:
model = keras.models.load_model('./insurance.h5')

### Descison boundaries for 2 dimensions

![Descison Boundaries for 2 Dimensions](https://djcordhose.github.io/ai/img/manning/nn-reg.png)

In [16]:
# a little sanity check, does it work at all?

# within this code, we expect Olli to be a green customer with a high prabability
# 0: red
# 1: green
# 2: yellow

olli_data = [100, 47, 10]

X = np.array([olli_data])
model.predict(X)

array([[0.00259071, 0.8607017 , 0.13670751]], dtype=float32)

## Converting our Keras model to the alternative high-level Estimator model

In [17]:
# https://cloud.google.com/blog/products/gcp/new-in-tensorflow-14-converting-a-keras-model-to-a-tensorflow-estimator
estimator_model = tf.keras.estimator.model_to_estimator(keras_model=model)

In [18]:
# it still works the same, with a different style of API, though
x = {"hidden1_input": X}
list(estimator_model.predict(input_fn=tf.estimator.inputs.numpy_input_fn(x, shuffle=False)))

[{'softmax': array([0.00259071, 0.8607017 , 0.13670751], dtype=float32)}]

## Preparing our model for serving
* https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md
* https://www.tensorflow.org/serving/serving_basic

In [19]:
!rm -rf tf

import os

export_path_base = 'tf'
version = 1
export_path = os.path.join(
      tf.compat.as_bytes(export_path_base),
      tf.compat.as_bytes(str(version)))

tf.keras.backend.set_learning_phase(0)
sess = tf.keras.backend.get_session()

classification_inputs = tf.saved_model.utils.build_tensor_info(model.input)
classification_outputs_scores = tf.saved_model.utils.build_tensor_info(model.output)

signature =  tf.saved_model.signature_def_utils.build_signature_def(
    inputs={'inputs': classification_inputs},
    outputs={'scores': classification_outputs_scores},
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)

builder = tf.saved_model.builder.SavedModelBuilder(export_path)
builder.add_meta_graph_and_variables(
      sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
           tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature
      })
builder.save()

b'tf\\1\\saved_model.pb'